In [1]:
!pip install --upgrade gspread gspread_dataframe oauth2client
import pandas as pd
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
from datetime import datetime, timedelta
import requests
import base64
import time
import random

In [2]:
# 🔑 Google Colab用認証フロー
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default

creds, _ = default()
gc = gspread.authorize(creds)

In [3]:
# 🎵 Spotify API連携設定
client_id = '581c1bdd9d464fef81ade0e46271e24e'
client_secret = '92dbe0318c194f6bbe863599f864d373'

class TokenManager:
    def __init__(self, cid, secret):
        self.client_id = cid
        self.client_secret = secret
        self.token = None
        self.token_expiry_time = datetime.now()

    def _get_new_token(self):
        auth = base64.b64encode(f"{self.client_id}:{self.client_secret}".encode()).decode()
        headers = {'Authorization': f'Basic {auth}'}
        data = {'grant_type': 'client_credentials'}
        try:
            r = requests.post('https://accounts.spotify.com/api/token', headers=headers, data=data)
            r.raise_for_status()
            token_data = r.json()
            self.token = token_data['access_token']
            expires_in = token_data.get('expires_in', 3600)
            self.token_expiry_time = datetime.now() + timedelta(seconds=expires_in - 300)
            print("✅ Spotifyアクセストークンを正常に取得・更新しました。")
        except requests.exceptions.RequestException as e:
            print(f"❌ トークン取得に失敗しました: {e}")
            self.token = None

    def get_token(self):
        if self.token is None or datetime.now() >= self.token_expiry_time:
            print("トークンが期限切れ、または存在しないため、リフレッシュします。")
            self._get_new_token()
        return self.token

    def force_refresh_token(self):
        print("401エラー検知。トークンを強制的にリフレッシュします。")
        self._get_new_token()

def make_spotify_request(url, token_manager, max_retries=5):
    for attempt in range(max_retries):
        try:
            token = token_manager.get_token()
            if not token:
                raise Exception("トークンが取得できません。")

            headers = {"Authorization": f"Bearer {token}"}
            r = requests.get(url, headers=headers, timeout=10)

            if r.status_code == 200:
                return r.json()
            elif r.status_code == 401:
                print(f"URL: {url}")
                token_manager.force_refresh_token()
                continue
            elif r.status_code == 429:
                wait_time = int(r.headers.get('Retry-After', 5))
                print(f"レート制限(429)のため、{wait_time}秒待機します。")
                time.sleep(wait_time)
            elif r.status_code >= 500:
                wait_time = (2 ** attempt) + random.uniform(0, 1)
                print(f"サーバーエラー({r.status_code}) リトライ {attempt + 1}。{wait_time:.2f}秒待機します。")
                time.sleep(wait_time)
            else:
                return None
        except requests.exceptions.RequestException as e:
            print(f"リクエスト例外発生: {e} リトライします...")
            wait_time = (2 ** attempt) + random.uniform(0, 1)
            time.sleep(wait_time)
    return None

def get_latest_track_info(artist_id, token_manager):
    # アルバム・シングルを取得
    url = f"https://api.spotify.com/v1/artists/{artist_id}/albums?include_groups=album,single&market=JP&limit=10"
    data = make_spotify_request(url, token_manager)
    if not data or not data.get('items'):
        return 'N/A', 'N/A'

    # リリース日でソートして最新のアルバムを取得
    latest_album = sorted(data['items'], key=lambda x: x['release_date'], reverse=True)[0]
    album_id = latest_album['id']

    # アルバムのトラックを取得
    tracks_url = f"https://api.spotify.com/v1/albums/{album_id}/tracks?limit=1"
    tracks_data = make_spotify_request(tracks_url, token_manager)

    if tracks_data and tracks_data.get('items'):
        track = tracks_data['items'][0]
        track_name = track['name']
        track_id = track['id']
        embed_link = f"https://open.spotify.com/embed/track/{track_id}"
        return track_name, embed_link

    return 'N/A', 'N/A'

def get_artist_image_url(artist_id, token_manager):
    url = f"https://api.spotify.com/v1/artists/{artist_id}"
    data = make_spotify_request(url, token_manager)
    if data and data.get('images'):
        return data['images'][0]['url']
    return 'N/A'


In [4]:
# 📥 最新2スナップショットの読み込み
sh = gc.open("IHC_Snapshot")
titles = sorted([ws.title for ws in sh.worksheets() if ws.title != 'Sheet1'])[-2:]
df_prev = get_as_dataframe(sh.worksheet(titles[0])).dropna(how='all').set_index('spotify_id')
df_now = get_as_dataframe(sh.worksheet(titles[1])).dropna(how='all').set_index('spotify_id')

print("===== 読み込んだシート名 =====")
print("前回スナップショット:", titles[0])
print("今回スナップショット:", titles[1])


===== 読み込んだシート名 =====
前回スナップショット: 2026-01-24
今回スナップショット: 2026-01-25


In [5]:
# 📊 スコア計算（トップ5曲の track_popularity_sum_ratio を使用）
df_diff = df_now.copy()

# df_prev にデータがない新規アーティストのインデックスを取得
new_artist_ids = df_now.index.difference(df_prev.index)

# 'name' 列を df_now から確実に引き継ぐ
df_diff['name'] = df_now['name']

# --- popularity_delta の計算修正 ---
# 新規アーティスト: 前回比較対象がないため、変化量は 0 とする
prev_popularity = df_prev.get('artist_popularity', pd.Series(dtype='float64')).reindex(df_now.index)
df_diff['popularity_delta'] = df_now['artist_popularity'] - prev_popularity
df_diff.loc[new_artist_ids, 'popularity_delta'] = 0
df_diff['popularity_delta'].fillna(0, inplace=True)


# --- followers_ratio の計算修正 ---
current_followers = df_now['followers']
prev_followers_series = df_prev.get('followers', pd.Series(dtype='float64')).reindex(df_now.index)
prev_followers_filled = prev_followers_series.fillna(0)
denominator_followers = prev_followers_filled.replace(0, 1)
df_diff['followers_ratio'] = (current_followers - prev_followers_filled) / denominator_followers
# 新規アーティストの followers_ratio を 0.0 に上書き
df_diff.loc[new_artist_ids, 'followers_ratio'] = 0.0


# --- track_popularity_sum_ratio の計算修正 ---
current_track_pop_sum = df_now['track_popularity_sum']
prev_track_pop_sum_series = df_prev.get('track_popularity_sum', pd.Series(dtype='float64')).reindex(df_now.index)
prev_track_pop_sum_filled = prev_track_pop_sum_series.fillna(0)
denominator_track_pop_sum = prev_track_pop_sum_filled.replace(0, 1)
df_diff['track_popularity_sum_ratio'] = (current_track_pop_sum - prev_track_pop_sum_filled) / denominator_track_pop_sum
# 新規アーティストの track_popularity_sum_ratio を 0.0 に上書き
df_diff.loc[new_artist_ids, 'track_popularity_sum_ratio'] = 0.0


# --- new_release_count の扱い ---
df_diff['new_release_count'] = df_now['new_release_count'] # まず全アーティストに現在の値をセット
# 「極力ランキングに影響が出ない」ことを最優先とする場合、
# 新規アーティストの new_release_count も 0 に設定します。
df_diff.loc[new_artist_ids, 'new_release_count'] = 0


# スコア計算
df_diff['score'] = (
    df_diff['popularity_delta'] * 3.0 +       # 新規アーティストは 0
    df_diff['followers_ratio'] * 2.0 +          # 新規アーティストは 0
    #df_diff['followers_ratio'] * 10.0 +          # 新規アーティストは 0  2025/10/11調整
    df_diff['track_popularity_sum_ratio'] * 1.0 + # 新規アーティストは 0
    df_diff['new_release_count'] * 0.3        # 新規アーティストは上記の処理により 0
    #df_diff['new_release_count'] * 0.1        # 新規アーティストは上記の処理により 0  2025/10/11調整
)

# --- ▼▼▼ ここから修正 ▼▼▼ ---
# artist_popularity が 0 のアーティストの score を 0 にする
# df_diff には df_now からコピーされた artist_popularity が存在する想定です。
# artist_popularity カラムを数値型に変換（エラー時はNaNにし、比較でFalseになるようにする）
df_diff['artist_popularity'] = pd.to_numeric(df_diff['artist_popularity'], errors='coerce')

# artist_popularity が 0 (またはNaN) の場合に条件を設定
condition_zero_popularity = (df_diff['artist_popularity'] == 0) | (df_diff['artist_popularity'].isna())

# 修正案：0 と 1 と 2、または NaN を除外対象にする
condition_low_popularity = df_diff['artist_popularity'].isin([0, 1, 2]) | df_diff['artist_popularity'].isna()
df_diff.loc[condition_low_popularity, 'score'] = 0

num_affected_artists = condition_zero_popularity.sum()
if num_affected_artists > 0:
    print(f"{num_affected_artists} 人のアーティストのスコアを0に調整しました (artist_popularityが0または無効値のため)。")
# --- ▲▲▲ ここまで修正 ▲▲▲ ---

ranking = df_diff.sort_values(by='score', ascending=False).reset_index()

74 人のアーティストのスコアを0に調整しました (artist_popularityが0または無効値のため)。


/tmp/ipython-input-3819332458.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_diff['popularity_delta'].fillna(0, inplace=True)


In [6]:
# 順位列を先頭に追加
ranking.insert(0, 'rank', ranking.index + 1)

# 列の並び替え
desired_order = [
    'rank',
    'score',
    'name',
    'spotify_id',
    'artist_popularity',
    'popularity_delta',
    'followers',
    'followers_ratio',
    'new_release_count',
    'track_popularity_sum_ratio',
    'track_popularity_sum'
]
# 指定順で DataFrame を再構築
ranking = ranking[desired_order]


In [7]:
######この処理はスプレッドシートの読み込みの失敗でエラーが出る場合があります。エラーが出た時は再実行してくださいい。######
# 📥 IHC_Ranking の最新シートから前回スコアを取得
sh_ranking = gc.open("IHC_Ranking")
latest_ws = sorted(sh_ranking.worksheets(), key=lambda ws: ws.title)[-1]
df_prev_score = get_as_dataframe(latest_ws).dropna(how="all")


# スプレッドシートを開いてすべてのシート名を表示
sh_ranking = gc.open("IHC_Ranking")
sheet_titles = [ws.title for ws in sh_ranking.worksheets()]
print("📄 シート一覧:", sheet_titles)

# ソートせず単に最後のシートを取得（作成順）
ws_latest = sh_ranking.worksheets()[-1]
print("✅ 最新シート名:", ws_latest.title)


from gspread_dataframe import get_as_dataframe

df_prev_score = get_as_dataframe(ws_latest).dropna(how="all")
#print("📊 データ先頭5行:")
#print(df_prev_score.head())

# 🔍 構造確認
print("📊 読み込んだカラム名:", df_prev_score.columns.tolist())



# ✅ spotify_id が存在するかチェックして index に設定
# IHC_Rankingシートのカラム名が日本語であることを想定
if "Spotify ID" in df_prev_score.columns and "スコア" in df_prev_score.columns and "順位" in df_prev_score.columns:
    df_prev_score = df_prev_score.set_index("Spotify ID")
    # 前回順位とスコアを抽出してカラム名を変更
    df_prev_score = df_prev_score[["順位", "スコア"]].rename(columns={"順位": "prev_rank", "スコア": "score_prev"})
    print("✅ 'prev_rank', 'score_prev' に変換済み")
else:
    # 英語カラム名の場合もフォールバックとしてチェック
    if "spotify_id" in df_prev_score.columns and "score" in df_prev_score.columns and "rank" in df_prev_score.columns:
        df_prev_score = df_prev_score.set_index("spotify_id")
        df_prev_score = df_prev_score[["rank", "score"]].rename(columns={"rank": "prev_rank", "score": "score_prev"})
        print("✅ 'prev_rank', 'score_prev' に変換済み (英語カラム名使用)")
    else:
        raise ValueError("❌ 'Spotify ID'/'spotify_id' または '順位'/'rank' または 'スコア'/'score' 列が存在しません。カラム名: " + str(df_prev_score.columns.tolist()))

# 🔗 現在のランキングとマージしてスコア差分を計算
# ranking DataFrame は英語カラム名 'spotify_id' を持つと想定
ranking = ranking.merge(df_prev_score, how="left", left_on="spotify_id", right_index=True)
ranking["score_delta"] = ranking["score"] - ranking["score_prev"]
# 前回順位のカラム名を日本語に設定
ranking.rename(columns={"prev_rank": "前回順位"}, inplace=True)


# 🔥 急上昇フラグとアイコンの付与
threshold = 5.0
ranking["rising_flag"] = ranking["score_delta"] > threshold
ranking["rising_icon"] = ranking["rising_flag"].apply(lambda x: "🔥" if x else "")

# 📑 カラムの並び替え（再定義）
desired_order = [
    'rank', '前回順位', 'score', 'score_prev', 'score_delta', 'rising_icon',
    'name', 'spotify_id', 'artist_popularity', 'popularity_delta',
    'followers', 'followers_ratio', 'new_release_count',
    'track_popularity_sum_ratio', 'track_popularity_sum'
]
ranking = ranking[desired_order]

📄 シート一覧: ['Sheet1', 'Ranking-2025-05-22', 'Ranking-2025-05-23', 'Ranking-2025-05-24', 'Ranking-2025-05-25', 'Ranking-2025-05-26', 'Ranking-2025-05-27', 'Ranking-2025-05-28', 'Ranking-2025-05-29', 'Ranking-2025-05-30', 'Ranking-2025-05-31', 'Ranking-2025-06-01', 'Ranking-2025-06-02', 'Ranking-2025-06-03', 'Ranking-2025-06-04', 'Ranking-2025-06-05', 'Ranking-2025-06-06', 'Ranking-2025-06-07', 'Ranking-2025-06-08', 'Ranking-2025-06-09', 'Ranking-2025-06-10', 'Ranking-2025-06-11', 'Ranking-2025-06-12', 'Ranking-2025-06-13', 'Ranking-2025-06-14', 'Ranking-2025-06-15', 'Ranking-2025-06-16', 'Ranking-2025-06-17', 'Ranking-2025-06-18', 'Ranking-2025-06-19', 'Ranking-2025-06-20', 'Ranking-2025-06-21', 'Ranking-2025-06-22', 'Ranking-2025-06-23', 'Ranking-2025-06-24', 'Ranking-2025-06-25', 'Ranking-2025-06-26', 'Ranking-2025-06-27', 'Ranking-2025-06-28', 'Ranking-2025-06-29', 'Ranking-2025-06-30', 'Ranking-2025-07-01', 'Ranking-2025-07-02', 'Ranking-2025-07-03', 'Ranking-2025-07-04', 'Ranking-202

In [8]:
# --- ここから追加統計情報作成処理 ---
print("\n--- 追加統計情報作成開始 ---")

# スプレッドシート名 (新規作成または既存のものを利用)
STATS_SPREADSHEET_NAME = "IHC_Daily_Stats"
USER_EMAIL_TO_SHARE = "yamagk2017@gmail.com" # 権限を付与するメールアドレス

# 1. 当日データの準備 (ranking DataFrame を使用)
# ranking DataFrame はセル7の処理で 'spotify_id', 'score', 'artist_popularity', 'track_popularity_sum_ratio' を持つ想定
df_today_stats_source = ranking.copy()
# 念のため、必要なカラムが数値型であることを確認
df_today_stats_source['artist_popularity'] = pd.to_numeric(df_today_stats_source['artist_popularity'], errors='coerce')
df_today_stats_source['track_popularity_sum_ratio'] = pd.to_numeric(df_today_stats_source['track_popularity_sum_ratio'], errors='coerce')
df_today_stats_source['score'] = pd.to_numeric(df_today_stats_source['score'], errors='coerce')


# 2. 前日データの準備
df_yesterday_stats_source = pd.DataFrame() # 空のDataFrameで初期化
# sh_ranking はセル7で gc.open("IHC_Ranking") されたものを利用
worksheets_in_ranking = sh_ranking.worksheets()
# 'Sheet1'のようなデフォルトシートを除外し、日付名でソートされた有効なランキングシートを取得
valid_ranking_sheets = sorted(
    [ws for ws in worksheets_in_ranking if ws.title != 'Sheet1' and 'Ranking-' in ws.title],
    key=lambda ws: ws.title
)

if len(valid_ranking_sheets) > 1:
    # 最新の一つ前のシートが前日のランキング
    ws_yesterday_ranking_title = valid_ranking_sheets[-1].title
    print(f"🔄 前日のランキングシート '{ws_yesterday_ranking_title}' を読み込みます。")
    df_yesterday_raw = get_as_dataframe(sh_ranking.worksheet(ws_yesterday_ranking_title)).dropna(how="all")

    # カラム名を英語に標準化 (日本語カラム名を想定)
    rename_map_jp_to_en = {
        "Spotify ID": "spotify_id",
        "人気度": "artist_popularity",
        "トップ5曲人気変化率": "track_popularity_sum_ratio",
        "スコア": "score",
    }
    rename_map_en = {col: col for col in ["spotify_id", "artist_popularity", "track_popularity_sum_ratio", "score"]}
    required_cols_for_stats_jp = ["人気度", "トップ5曲人気変化率", "スコア"]
    required_cols_for_stats_en = ["artist_popularity", "track_popularity_sum_ratio", "score"]

    if all(col in df_yesterday_raw.columns for col in required_cols_for_stats_jp) and "Spotify ID" in df_yesterday_raw.columns:
        df_yesterday_stats_source = df_yesterday_raw.rename(columns=rename_map_jp_to_en)
        print("    前日データは日本語カラム名でした。英語に変換しました。")
    elif all(col in df_yesterday_raw.columns for col in required_cols_for_stats_en) and "spotify_id" in df_yesterday_raw.columns:
        df_yesterday_stats_source = df_yesterday_raw.rename(columns=rename_map_en)
        print("    前日データは英語カラム名でした。")
    else:
        print(f"⚠️ 前日のランキングシート '{ws_yesterday_ranking_title}' に必要なカラムが見つかりません。")
        print(f"    見つかったカラム: {df_yesterday_raw.columns.tolist()}")
        df_yesterday_stats_source = pd.DataFrame()

    if not df_yesterday_stats_source.empty:
        for col in ['artist_popularity', 'track_popularity_sum_ratio', 'score']:
            if col in df_yesterday_stats_source.columns:
                 df_yesterday_stats_source[col] = pd.to_numeric(df_yesterday_stats_source[col], errors='coerce')
            else:
                print(f"⚠️ 前日データにカラム '{col}' がありません。この指標の前日値は計算できません。")
                df_yesterday_stats_source[col] = pd.NA
elif len(valid_ranking_sheets) == 1:
    print("📜 ランキング履歴が1シートのみです。前日データなしとして扱います。")
else:
    print("📜 有効なランキング履歴が見つかりません。前日データなしとして扱います。")

# 3. 指標計算 (当日分)
count_pop_zero_today = df_today_stats_source[df_today_stats_source['artist_popularity'] == 0].shape[0] if not df_today_stats_source.empty else 0
count_tpsr_zero_today = df_today_stats_source[df_today_stats_source['track_popularity_sum_ratio'] == 0].shape[0] if not df_today_stats_source.empty else 0
count_both_zero_today = df_today_stats_source[(df_today_stats_source['artist_popularity'] == 0) & (df_today_stats_source['track_popularity_sum_ratio'] == 0)].shape[0] if not df_today_stats_source.empty else 0
avg_score_today = df_today_stats_source['score'].mean() if not df_today_stats_source.empty and 'score' in df_today_stats_source.columns and not df_today_stats_source['score'].dropna().empty else pd.NA

# 指標計算 (前日分)
count_pop_zero_yesterday, count_tpsr_zero_yesterday, count_both_zero_yesterday, avg_score_yesterday = pd.NA, pd.NA, pd.NA, pd.NA
if not df_yesterday_stats_source.empty:
    if 'artist_popularity' in df_yesterday_stats_source.columns:
        count_pop_zero_yesterday = df_yesterday_stats_source[df_yesterday_stats_source['artist_popularity'] == 0].shape[0]
    if 'track_popularity_sum_ratio' in df_yesterday_stats_source.columns:
        count_tpsr_zero_yesterday = df_yesterday_stats_source[df_yesterday_stats_source['track_popularity_sum_ratio'] == 0].shape[0]
    if 'artist_popularity' in df_yesterday_stats_source.columns and 'track_popularity_sum_ratio' in df_yesterday_stats_source.columns:
        count_both_zero_yesterday = df_yesterday_stats_source[(df_yesterday_stats_source['artist_popularity'] == 0) & (df_yesterday_stats_source['track_popularity_sum_ratio'] == 0)].shape[0]
    if 'score' in df_yesterday_stats_source.columns and not df_yesterday_stats_source['score'].dropna().empty:
        avg_score_yesterday = df_yesterday_stats_source['score'].mean()
else:
    print("ℹ️ 前日の統計指標の計算に必要なデータが存在しません。")

# 4. 前日比計算関数
def calculate_change_stats(today_val, yesterday_val):
    diff, ratio_str = pd.NA, "N/A"
    if pd.notna(today_val) and pd.notna(yesterday_val):
        diff = today_val - yesterday_val
        if yesterday_val != 0: ratio_str = f"{(diff / yesterday_val) * 100:.2f}%"
        elif today_val == 0: ratio_str = "0.00%"
        else: ratio_str = "N/A (prev 0)"
    elif pd.notna(today_val): diff = today_val; ratio_str = "N/A (no prev)"
    elif pd.notna(yesterday_val): diff = -yesterday_val; ratio_str = "N/A (no today)"
    return diff, ratio_str

pop_zero_diff, pop_zero_ratio_str = calculate_change_stats(count_pop_zero_today, count_pop_zero_yesterday)
tpsr_zero_diff, tpsr_zero_ratio_str = calculate_change_stats(count_tpsr_zero_today, count_tpsr_zero_yesterday)
both_zero_diff, both_zero_ratio_str = calculate_change_stats(count_both_zero_today, count_both_zero_yesterday)
avg_score_diff, avg_score_ratio_str = calculate_change_stats(avg_score_today, avg_score_yesterday)

# 5. 出力用DataFrame作成
stats_data = {
    "指標名": [
        "全アーティストの score 平均値",                                       # ← 先頭に移動
        "artist_popularity が 0",
        "track_popularity_sum_ratio が 0",
        "両方が 0"  # ノートブックの表記に合わせる
    ],
    "当日値": [
        f"{avg_score_today:.2f}" if pd.notna(avg_score_today) else "N/A",    # ← 先頭に移動
        count_pop_zero_today,
        count_tpsr_zero_today,
        count_both_zero_today
    ],
    "前日値": [
        f"{avg_score_yesterday:.2f}" if pd.notna(avg_score_yesterday) else "N/A", # ← 先頭に移動
        count_pop_zero_yesterday if pd.notna(count_pop_zero_yesterday) else "N/A",
        count_tpsr_zero_yesterday if pd.notna(count_tpsr_zero_yesterday) else "N/A",
        count_both_zero_yesterday if pd.notna(count_both_zero_yesterday) else "N/A"
    ],
    "前日差": [
        f"{avg_score_diff:.2f}" if pd.notna(avg_score_diff) else "N/A",    # ← 先頭に移動
        pop_zero_diff if pd.notna(pop_zero_diff) else "N/A",
        tpsr_zero_diff if pd.notna(tpsr_zero_diff) else "N/A",
        both_zero_diff if pd.notna(both_zero_diff) else "N/A"
    ],
    "前日比": [
        avg_score_ratio_str,                                               # ← 先頭に移動
        pop_zero_ratio_str,
        tpsr_zero_ratio_str,
        both_zero_ratio_str
    ]
}
df_output_stats = pd.DataFrame(stats_data)

# 6. スプレッドシートへの出力
try:
    sh_stats = gc.open(STATS_SPREADSHEET_NAME)
    print(f"📄 既存のスプレッドシート '{STATS_SPREADSHEET_NAME}' を開きました。")
except gspread.exceptions.SpreadsheetNotFound:
    sh_stats = gc.create(STATS_SPREADSHEET_NAME)
    print(f"🎉 新規スプレッドシート '{STATS_SPREADSHEET_NAME}' を作成しました。")
    if len(sh_stats.worksheets()) > 0 and sh_stats.worksheets()[0].title == 'Sheet1' and len(df_output_stats) > 0:
        try:
            if len(sh_stats.worksheets()) == 1 :
                 sh_stats.add_worksheet(title="temp_sheet_for_deletion", rows="1", cols="1") # ダミーシート追加
            sh_stats.del_worksheet(sh_stats.worksheet('Sheet1'))
            print("    デフォルトシート 'Sheet1' を削除しました。")
        except Exception as e:
            print(f"    デフォルトシート 'Sheet1' の削除に失敗しました: {e}")

stats_sheet_name_base = datetime.today().strftime("Stats-%Y-%m-%d")
existing_stat_sheet_names = [ws.title for ws in sh_stats.worksheets()]
i_stats = 1
final_stats_sheet_name = stats_sheet_name_base
while final_stats_sheet_name in existing_stat_sheet_names:
    final_stats_sheet_name = f"{stats_sheet_name_base}-{i_stats}"
    i_stats += 1

ws_stats = sh_stats.add_worksheet(title=final_stats_sheet_name, rows=str(len(df_output_stats) + 5), cols=str(len(df_output_stats.columns) + 2 ))
set_with_dataframe(ws_stats, df_output_stats, include_index=False, include_column_header=True, resize=True)
print(f"✅ 統計情報出力完了: {sh_stats.url}（シート名: {final_stats_sheet_name}）")

# --- ここから権限付与処理 ---
if USER_EMAIL_TO_SHARE:
    try:
        sh_stats.share(USER_EMAIL_TO_SHARE, perm_type='user', role='writer', notify=False) # notify=False でメール通知をオフ
        print(f"🔒 スプレッドシート '{STATS_SPREADSHEET_NAME}' に {USER_EMAIL_TO_SHARE} の編集権限を付与しました。")
    except Exception as e:
        print(f"⚠️ {USER_EMAIL_TO_SHARE} への権限の付与に失敗しました: {e}")
# --- 権限付与処理ここまで ---

print("--- 追加統計情報作成終了 ---")
# --- 追加処理ここまで ---


--- 追加統計情報作成開始 ---
🔄 前日のランキングシート 'Ranking-2026-01-24' を読み込みます。
    前日データは英語カラム名でした。
📄 既存のスプレッドシート 'IHC_Daily_Stats' を開きました。
✅ 統計情報出力完了: https://docs.google.com/spreadsheets/d/1mwoYZ0AQJadMi_emJuS-_zIrvTXKvnPbhEqiuhpPW1o（シート名: Stats-2026-01-25）
🔒 スプレッドシート 'IHC_Daily_Stats' に yamagk2017@gmail.com の編集権限を付与しました。
--- 追加統計情報作成終了 ---


In [9]:
# 📤 ランキングをスプレッドシートに書き出し（履歴蓄積）

# 日本語カラム名への変換
ranking_output = ranking.copy()  # 出力用にコピー

# まず score 列を10倍（まだ英語列名）
ranking_output["score"] = ranking_output["score"] * 10

# それから rename
ranking_output = ranking_output.rename(columns={
    "rank": "順位",
    "score": "スコア",
    "score_prev": "前回スコア",
    "score_delta": "スコア差分",
    "rising_icon": "急上昇",
    "name": "アーティスト名",
    "spotify_id": "Spotify ID",
    "artist_popularity": "人気度",
    "popularity_delta": "人気度変化",
    "followers": "フォロワー数",
    "followers_ratio": "フォロワー増加率",
    "new_release_count": "新曲数",
    "track_popularity_sum_ratio": "トップ5曲人気変化率",
    "track_popularity_sum": "トップ5曲人気合計"
})

try:
    sh_out = gc.open("IHC_Ranking")
except gspread.SpreadsheetNotFound:
    sh_out = gc.create("IHC_Ranking")

base = datetime.today().strftime("Ranking-%Y-%m-%d")
names = [ws.title for ws in sh_out.worksheets()]
new_name = base
i = 1
while new_name in names:
    new_name = f"{base}-{i}"
    i += 1

ws = sh_out.add_worksheet(
    title=new_name,
    rows=str(len(ranking_output) + 1),
    cols="20"
)
set_with_dataframe(ws, ranking_output)
print(f"✅ ランキング出力完了: {sh_out.url}（シート名: {new_name}）")


✅ ランキング出力完了: https://docs.google.com/spreadsheets/d/1kkJNnn4pZ_DG5_5j6sfH5HtdWblAh88zF37KhIhCn7g（シート名: Ranking-2026-01-25）


In [10]:
# --- ここから積算スコアランキング作成処理 ---
print("\n--- 積算スコアランキング作成開始 ---")

# 1. 新しいスプレッドシート名（またはID）を指定
cumulative_ranking_spreadsheet_name = "IHC_Cumulative_Ranking"

try:
    sh_cumulative = gc.open(cumulative_ranking_spreadsheet_name)
    print(f"📄 既存のスプレッドシート '{cumulative_ranking_spreadsheet_name}' を開きました。")
except gspread.exceptions.SpreadsheetNotFound:
    sh_cumulative = gc.create(cumulative_ranking_spreadsheet_name)
    print(f"🎉 新規スプレッドシート '{cumulative_ranking_spreadsheet_name}' を作成しました。")

# 2. 前回の積算スコアを読み込む
df_prev_cumulative_score = pd.DataFrame(columns=['spotify_id', 'cumulative_score_prev']) # デフォルト空DF

cumulative_worksheets = sh_cumulative.worksheets()
# 'Sheet1'のようなデフォルトシートを除外し、日付名でソートして最新を取得
valid_cumulative_sheets = sorted([ws for ws in cumulative_worksheets if ws.title != 'Sheet1' and 'Ranking-' in ws.title], key=lambda ws: ws.title)

if valid_cumulative_sheets:
    latest_cumulative_ws_title = valid_cumulative_sheets[-1].title
    print(f"🔄 前回の積算ランキングシート '{latest_cumulative_ws_title}' を読み込みます。")
    df_prev_cumulative_score_temp = get_as_dataframe(sh_cumulative.worksheet(latest_cumulative_ws_title)).dropna(how="all")
    # 前回のシートのカラム名が日本語であることを想定
    if "Spotify ID" in df_prev_cumulative_score_temp.columns and "積算スコア" in df_prev_cumulative_score_temp.columns:
         df_prev_cumulative_score = df_prev_cumulative_score_temp[["Spotify ID", "積算スコア"]].rename(columns={"Spotify ID": "spotify_id", "積算スコア": "cumulative_score_prev"})
    elif "spotify_id" in df_prev_cumulative_score_temp.columns and "cumulative_score" in df_prev_cumulative_score_temp.columns: # 英語カラム名の場合もフォールバック
         df_prev_cumulative_score = df_prev_cumulative_score_temp[["spotify_id", "cumulative_score"]].rename(columns={"cumulative_score": "cumulative_score_prev"})
         print("    (前回の積算シートは英語カラム名でした)")
    else:
        print(f"⚠️ 前回の積算ランキングシート '{latest_cumulative_ws_title}' に必要カラム（Spotify ID, 積算スコア）が見つかりません。積算スコアは0から開始します。")
else:
    print("📜 積算ランキングの履歴が見つかりません。初回実行として積算スコアは0から開始します。")

# 3. 今回のスコアを取得 (ranking_output DataFrameは日本語カラム名になっている)
# ranking_output から 'Spotify ID' と 'スコア' を取得し、内部処理用に英語カラム名に戻す
if 'Spotify ID' in ranking_output.columns and 'スコア' in ranking_output.columns:
    df_current_scores = ranking_output[['Spotify ID', 'スコア']].rename(columns={"Spotify ID": "spotify_id", "スコア": "score"})
else:
    print("❌ ranking_output DataFrameに 'Spotify ID' または 'スコア' が見つかりません。積算処理をスキップします。")
    df_current_scores = pd.DataFrame(columns=['spotify_id', 'score']) # 空のDFで処理を継続できるようにする

# 4. 積算スコアを計算
if not df_current_scores.empty:
    df_cumulative_merged = pd.merge(df_current_scores, df_prev_cumulative_score, on='spotify_id', how='left')
    df_cumulative_merged['cumulative_score_prev'] = pd.to_numeric(df_cumulative_merged['cumulative_score_prev'], errors='coerce').fillna(0).astype(float)
    df_cumulative_merged['score'] = pd.to_numeric(df_cumulative_merged['score'], errors='coerce').fillna(0).astype(float)
    df_cumulative_merged['cumulative_score'] = df_cumulative_merged['cumulative_score_prev'] + df_cumulative_merged['score']

    # 5. ランキング作成に必要な情報をマージ (アーティスト名、人気度など)
    # ranking_output から 'Spotify ID', 'アーティスト名', '人気度' を取得し、内部処理用に英語カラム名に戻す
    # '人気度' カラムが ranking_output に存在することを想定
    required_info_columns = ['Spotify ID', 'アーティスト名', '人気度']
    if all(col in ranking_output.columns for col in required_info_columns):
        df_artist_info = ranking_output[required_info_columns].rename(
            columns={"Spotify ID": "spotify_id", "アーティスト名": "name", "人気度": "artist_popularity_current"}
        ).drop_duplicates(subset=['spotify_id'])
        df_final_cumulative_ranking = pd.merge(df_cumulative_merged[['spotify_id', 'cumulative_score', 'score']], df_artist_info, on='spotify_id', how='left')
    elif 'Spotify ID' in ranking_output.columns and 'アーティスト名' in ranking_output.columns:
        print("⚠️ ranking_output DataFrameに '人気度' カラムが見つかりませんでした。アーティスト名のみマージし、人気度はNAとします。")
        df_artist_names = ranking_output[['Spotify ID', 'アーティスト名']].rename(
            columns={"Spotify ID": "spotify_id", "アーティスト名": "name"}
        ).drop_duplicates(subset=['spotify_id'])
        df_final_cumulative_ranking = pd.merge(df_cumulative_merged[['spotify_id', 'cumulative_score', 'score']], df_artist_names, on='spotify_id', how='left')
        df_final_cumulative_ranking['artist_popularity_current'] = pd.NA # 人気度カラム用のプレースホルダー
    else:
        print("❌ ranking_output DataFrameに 'Spotify ID' または 'アーティスト名' が見つかりません。アーティスト名・人気度なしで処理を続行します。")
        df_final_cumulative_ranking = df_cumulative_merged[['spotify_id', 'cumulative_score', 'score']].copy()
        df_final_cumulative_ranking['name'] = 'N/A'
        df_final_cumulative_ranking['artist_popularity_current'] = pd.NA


    # 6. 積算スコアでソートし、順位を付与
    df_final_cumulative_ranking = df_final_cumulative_ranking.sort_values(by='cumulative_score', ascending=False).reset_index(drop=True)
    df_final_cumulative_ranking.insert(0, 'rank', df_final_cumulative_ranking.index + 1)

    # 7. カラム名を整え、出力用に日本語に戻す
    df_final_cumulative_ranking = df_final_cumulative_ranking.rename(columns={
        "rank": "順位",
        "cumulative_score": "積算スコア",
        "score": "今回獲得スコア",
        "name": "アーティスト名",
        "spotify_id": "Spotify ID",
        "artist_popularity_current": "集計時人気度" # artist_popularity を日本語カラム名に
    })
    # 出力するカラムの順序を定義（新しいカラム「集計時人気度」を追加）
    output_columns_cumulative = ["順位", "積算スコア", "今回獲得スコア", "アーティスト名", "Spotify ID", "集計時人気度"]

    # df_final_cumulative_rankingに存在するカラムのみを選択 (エラー回避のため)
    output_columns_cumulative = [col for col in output_columns_cumulative if col in df_final_cumulative_ranking.columns]
    df_final_cumulative_ranking = df_final_cumulative_ranking[output_columns_cumulative]

    # 8. スプレッドシートに書き出し
    base_cumulative = datetime.today().strftime("Ranking-%Y-%m-%d")
    names_cumulative = [ws.title for ws in sh_cumulative.worksheets()]
    new_name_cumulative = base_cumulative
    i_cumulative = 1
    while new_name_cumulative in names_cumulative:
        new_name_cumulative = f"{base_cumulative}-{i_cumulative}"
        i_cumulative += 1

    # 新しいシートの列数を、出力するカラム数に合わせて調整
    ws_cumulative = sh_cumulative.add_worksheet(title=new_name_cumulative, rows=str(len(df_final_cumulative_ranking) + 1), cols=str(len(output_columns_cumulative) + 1))
    set_with_dataframe(ws_cumulative, df_final_cumulative_ranking)
    print(f"✅ 積算スコアランキング出力完了: {sh_cumulative.url}（シート名: {new_name_cumulative}）")
else:
    print("ℹ️ 今回のスコアデータが空のため、積算スコアランキングの作成をスキップしました。")

print("--- 積算スコアランキング作成終了 ---")
# --- 積算スコアランキング作成処理ここまで ---


--- 積算スコアランキング作成開始 ---
📄 既存のスプレッドシート 'IHC_Cumulative_Ranking' を開きました。
🔄 前回の積算ランキングシート 'Ranking-2026-01-24' を読み込みます。
✅ 積算スコアランキング出力完了: https://docs.google.com/spreadsheets/d/11OJNuEj4s37KGAfKvljCka_NwJ8zus8w5co5-lNVEdQ（シート名: Ranking-2026-01-25）
--- 積算スコアランキング作成終了 ---


In [11]:
# 🚀 TOP20ランキングを専用スプレッドシートに出力
print("\n--- TOP20ランキング作成開始 ---")

# 1. Spotify APIの準備
token_manager = TokenManager(client_id, client_secret)

# 2. TOP20アーティストの選出
top20_df = ranking.head(20).copy()

# 3. 最新楽曲と画像URLの情報を取得
extra_info = []
for artist_id in top20_df['spotify_id']:
    print(f"🎵 {artist_id} の情報を取得中...")
    track_name, embed_link = get_latest_track_info(artist_id, token_manager)
    image_url = get_artist_image_url(artist_id, token_manager)
    extra_info.append({
        'spotify_id': artist_id,
        'latest_track_name': track_name,
        'latest_track_embed_link': embed_link,
        'artist_image_url': image_url
    })
    time.sleep(0.5) # APIへの負荷軽減

df_extra_info = pd.DataFrame(extra_info)
top20_df = pd.merge(top20_df, df_extra_info, on='spotify_id')

# 4. データ整形
top20_output = pd.DataFrame()
top20_output['順位'] = top20_df['rank'].astype(str) + '. '
top20_output['アーティスト名'] = top20_df['name']
top20_output['ポイント'] = (top20_df['score'] * 10).round(1).astype(str) + 'pt'
top20_output['最新楽曲名'] = top20_df['latest_track_name']
top20_output['埋め込みリンク'] = top20_df['latest_track_embed_link']
top20_output['アーティスト画像URL'] = top20_df['artist_image_url']

# 5. スプレッドシートへの書き込み
TOP20_SHEET_NAME = "IHC_Top20_Daily"
USER_EMAIL_TO_SHARE_TOP20 = "yamadaclear@gmail.com"
try:
    sh_top20 = gc.open(TOP20_SHEET_NAME)
    print(f"📄 既存のスプレッドシート '{TOP20_SHEET_NAME}' を開きました。")
except gspread.exceptions.SpreadsheetNotFound:
    sh_top20 = gc.create(TOP20_SHEET_NAME)
    print(f"🎉 新規スプレッドシート '{TOP20_SHEET_NAME}' を作成しました。")

today_str = datetime.today().strftime('%Y-%m-%d')
try:
    worksheet_to_delete = sh_top20.worksheet(today_str)
    sh_top20.del_worksheet(worksheet_to_delete)
    print(f"🗑️ 既存のシート '{today_str}' を削除しました。")
except gspread.exceptions.WorksheetNotFound:
    pass

ws_top20 = sh_top20.add_worksheet(title=today_str, rows=str(len(top20_output) + 1), cols=str(len(top20_output.columns) + 1))
set_with_dataframe(ws_top20, top20_output, include_index=False, include_column_header=True, resize=True)
print(f"✅ TOP20ランキング出力完了: {sh_top20.url}（シート名: {today_str}）")

# 6. 権限付与
if USER_EMAIL_TO_SHARE_TOP20:
    try:
        sh_top20.share(USER_EMAIL_TO_SHARE_TOP20, perm_type='user', role='writer', notify=False)
        print(f"🔒 スプレッドシート '{TOP20_SHEET_NAME}' に {USER_EMAIL_TO_SHARE_TOP20} の編集権限を付与しました。")
    except Exception as e:
        print(f"⚠️ {USER_EMAIL_TO_SHARE_TOP20} への権限の付与に失敗しました: {e}")

print("--- TOP20ランキング作成終了 ---")



--- TOP20ランキング作成開始 ---
🎵 3MGnwZKDzNXiDtf1wMEmb0 の情報を取得中...
トークンが期限切れ、または存在しないため、リフレッシュします。
✅ Spotifyアクセストークンを正常に取得・更新しました。
🎵 0mIGl08cz5R5alv9ei0vNX の情報を取得中...
🎵 1sDZEQNXyRK7I7HcQ5md1y の情報を取得中...
🎵 1qls7cxRI0qSWA7D8PCA96 の情報を取得中...
🎵 5Woiwy2VjAWXM0DqQ107eF の情報を取得中...
🎵 5LMx3m0wCQgxBxbpPWXSFH の情報を取得中...
🎵 3t4frs22khzq3i1W8jvKX7 の情報を取得中...
🎵 2iVoT3dvAtPe7JIcDeFI3K の情報を取得中...
🎵 0Qwj2olxAhXpo1OVixtcRm の情報を取得中...
🎵 0Rd29hL7lZ1yNpzdt79o7C の情報を取得中...
🎵 0eQSoTI7sQENREQM8Klp2j の情報を取得中...
🎵 5bFJ3V4SnK7PeqFKZk1Zep の情報を取得中...
🎵 1koolUhp64aJl6lWUVTORj の情報を取得中...
🎵 2x8zQTiCA44QFI43j28VXN の情報を取得中...
🎵 5b30xP6EZF4QIVVvXiUmo3 の情報を取得中...
🎵 5kRsLXG9ruqeqweyVzxq8N の情報を取得中...
🎵 6U39AV2r0zGJfhsflAuVvG の情報を取得中...
🎵 754irAyZSIxQ3lGywEi24z の情報を取得中...
🎵 60SU8psYiKMEDRelpTaxxI の情報を取得中...
🎵 0HRL557om24yW4pH5hgmhs の情報を取得中...
📄 既存のスプレッドシート 'IHC_Top20_Daily' を開きました。
✅ TOP20ランキング出力完了: https://docs.google.com/spreadsheets/d/1mn7_JJRpR8IGhcd-Wpi9qDpTLkOCjhNhiaJx0HsYPxo（シート名: 2026-01-25）
🔒 スプレッドシート 'IHC_Top20_Daily' に y